In [5]:
import numpy as np 
import matplotlib as plt
import pandas as pd
import seaborn as sns

In [8]:
penguins_df = pd.read_csv("noisy_datasets/penguins.csv")
diamonds_df = pd.read_csv("noisy_datasets/diamond.csv")
epicurious_df = pd.read_csv("noisy_datasets/epicurious.csv")

penguins_df.head()

,species,island,calorie requirement,average sleep duration,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,gender,year
0,Adelie,Torgersen,6563,11,39.1,18.7,181.0,3750.0,male,2007.0
1,Adelie,Torgersen,4890,14,39.5,17.4,186.0,3800.0,female,2007.0
2,Adelie,Torgersen,7184,11,40.3,18.0,195.0,3250.0,female,2007.0
3,Adelie,Torgersen,4169,8,NaN,NaN,NaN,992.0,NaN,2007.0
4,Adelie,Torgersen,4774,8,36.7,19.3,193.0,3450.0,female,2007.0
